## Two simple schedules

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code


from fibertree_bootstrap import *
fibertree_bootstrap(style="tree", animation='movie', logger=False)


## Create a frontier

Create a `frontier` tensor to be processed in parallel

In [ ]:
f = Tensor.fromUncompressed(name="frontier",
                            rank_ids=["V"],
                            root=[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
displayTensor(f)



In [ ]:
# Set the number of PEs

PEs = 4

## **Dealing** - Parallel scheduling - type 1 

Divide the work by the number of PEs then process each chunk in order.

If one thinks of the `frontier` as a deck of cards, then this schedule is **dealing** out a sequence of *hands* of exactly one card to `PE` players and after each hand is dealt all the players play their *hand*.

If **slip** is implemented in this schedule then it manifests as a player going on to the next hand ahead of the other players... This is not illustrated yet.

In [ ]:
st1 = f.splitEqual(PEs)
displayTensor(st1)

### Dealing Animation

Below is an animation of the **dealing** schedule. Note how load imbalance results because not all the players have a card in the last hand, so two PEs are idle in the last cycle.

In [ ]:
# Show the parallelism

canvas = createCanvas(f, st1)

for v1_pos, (v1, st1_v0) in enumerate(st1):
    for v0_pos, (v0, _) in enumerate(st1_v0):
        canvas.addActivity((v0,), (v1, v0), spacetime=(v0_pos, v1_pos))
        
displayCanvas(canvas)

## **Cutting** - Parallel scheduling - type 2

Schedule that divides the work and then processes the sequenntially in each fiber. To implement that in the fibertree language we have to do a series of manipulations of the frontier...


If one thinks of the `frontier` as a deck of cards, then this schedule is **cutting** the deck into `PE` pieces and each of the `PE` players steps sequentially through their **cut** of the deck. 

If **slip** is implemented in this schedule then it manifests as each player going through their cut at their own pace.... This is not illustrated yet.

In [ ]:
import math

partition_size = math.ceil(len(f.getRoot()) / PEs)

print(f"Partition size: {partition_size}\n")

# Partition (but use relative coordinates, so the swap works)
st2_split = f.splitEqual(partition_size, relativeCoords=True)
print("\n\nSchedule split - note repeated coordinates in lower rank")
displayTensor(st2_split)

# Swap the ranks so the parallel work appears in a single fiber
st2 = st2_split.swapRanks()
print("\n\nSchedule swapped - note again the repeated coordintes in lower rank")
displayTensor(st2)

# Restore to absolute coordinates
for v0, st2_v1 in st2:
    st2_v1.updateCoords(lambda n, c, p: v0 + c)
    
print("\n\nFinal schedule - with original coordinates")
displayTensor(st2)

### Cutting animation

Below is an animation of the **cutting** scheudule. Note how load impbance results from some **cuts** belong smaller than others, so that in this case one PE is idle for two cycles at the end.


In [ ]:
# Show the parallelism

canvas = createCanvas(f, st2)

for v0_pos, (v0, st2_v1) in enumerate(st2):
    for v1_pos, (v1, _) in enumerate(st2_v1):
        canvas.addActivity((v1,), (v0, v1), spacetime=(v1_pos, v0_pos))
        
displayCanvas(canvas)